In [ ]:
pip install torch torchvisionpip install cython
pip install 'git+https://github.com/facebookresearch/fvcore.git'
pip install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
pip install detectron2

import torch
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.model_zoo import model_zoo

def setup_cfg():
    # Load a pre-trained HRNet model from Detectron2 model zoo
    cfg = get_cfg()
    cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/hrnet/w32_1x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Set the threshold for detection
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/hrnet/w32_1x.yaml")
    cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"  # Use GPU if available

    return cfg

def detect_objects(image_path, cfg):
    # Create a predictor using the configuration
    predictor = DefaultPredictor(cfg)

    # Load the input image
    image = torch.as_tensor(image_path)

    # Perform inference on the image
    outputs = predictor(image)

    # Visualize the predictions
    v = Visualizer(image[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    result_image = v.get_image()[:, :, ::-1]

    return result_image

if __name__ == "__main__":
    # Setup the configuration
    cfg = setup_cfg()

    # Path to the input image
    image_path = "path/to/your/image.jpg"

    # Perform object detection
    result_image = detect_objects(image_path, cfg)

    # Display the result
    import matplotlib.pyplot as plt

    plt.imshow(result_image)
    plt.axis("off")
    plt.show()
